Dự đoán phần trăm pin thu được vào lúc 18h.  
Có 2 phương hướng triển khai: xây model hoặc phân tích tay.  
Trong 2 ngày xây model 4-5/8 thì gặp 1 số vấn đề: data chưa bao quát hết tất cả các trường hợp, bị overfit.  
Quyết định notebook này sẽ triển khai theo hướng 2.  

1. Import thư viện

In [ ]:
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error


2. Import data

In [ ]:
# Đọc dữ liệu
df = pd.read_csv('device_log.csv')  

# Chuyển đổi sang datetime
df['ct'] = pd.to_datetime(df['ct'], unit='s')
df = df.sort_values(by=['device', 'ct'])

# Lấy ngày, giờ, phút
df['date'] = df['ct'].dt.date
df['hour'] = df['ct'].dt.hour
df['minute'] = df['ct'].dt.minute

# --- Lọc ngày đủ thời gian theo từng device và ngày ---
# Giữ lại những device, date mà dữ liệu đủ điều kiện: giờ min <= 6 và giờ max >= 18
valid_device_dates = df.groupby(['device', 'date']).filter(
    lambda x: (x['hour'].min() <= 6) and (x['hour'].max() >= 18)
)[['device', 'date']].drop_duplicates()

df = df.merge(valid_device_dates, on=['device', 'date'], how='inner')

# Lọc giờ trong khoảng 6h-19h để tránh bị những giờ k sạc ảnh hưởng tới việc phân tích hệ số
df = df[(df['hour'] >= 6) & (df['hour'] <= 19)]

# Tìm giá trị percentlt gần nhất 6h sáng của từng device, từng ngày
df_6h = df.loc[(df['hour'] == 6)].copy()
df_6h = df_6h.sort_values(['device', 'date', 'minute'])
df_6h = df_6h.groupby(['device', 'date']).first().reset_index()[['device', 'date', 'percentlt']]
df_6h = df_6h.rename(columns={'percentlt': 'percentlt_at_6h'})

# Ghép vào df chính
df = df.merge(df_6h, on=['device', 'date'], how='left')

# Tạo cột plus_percentlt
df['plus_percentlt'] = df['percentlt'] - df['percentlt_at_6h']

# Tính phút từ đầu ngày để tiện tính khoảng thời gian
df['minute_of_day'] = df['hour'] * 60 + df['minute']

# Tính phút từ 6h sáng
df['minutes_from_6h'] = (df['hour'] - 6) * 60 + df['minute']
df['minutes_from_6h'] = df['minutes_from_6h'].clip(lower=0)

# Tính delta_minute theo từng device riêng biệt
df = df.sort_values(['device', 'ct'])
df['delta_minute'] = df.groupby('device')['minutes_from_6h'].diff().fillna(0)
df.loc[df['delta_minute'] < 0, 'delta_minute'] = 0  # Nếu âm thì gán 0

# Đánh số nhóm liên tiếp theo weather_icon (theo toàn bộ df)
df['weather_group'] = (df['weather_icon'] != df['weather_icon'].shift()).cumsum()

# --- Lọc bỏ các thời điểm sau khi percentlt đạt 100% lần đầu tiên trong ngày theo từng device --- để tránh các phần percentlt giảm sau khi đầy (những đoạn mà giả định rằng data sai)
def filter_after_full_charge(group):
    full_charge_idx = group.index[group['percentlt'] >= 100]
    if len(full_charge_idx) == 0:
        return group
    first_full_charge_idx = full_charge_idx[0]
    return group.loc[:first_full_charge_idx]

df = df.groupby(['device', 'date'], group_keys=False).apply(filter_after_full_charge)

# Lưu kết quả
df.to_csv('train/device_log_processed.csv', index=False)

Huấn luyện model dự đoán psun theo: thời gian + weather_icon

In [ ]:
# --- Bước 1: Đọc dữ liệu đầy đủ
df = pd.read_csv('train/device_log_processed.csv')

# --- Bước 2: Lấy danh sách device, sắp xếp
devices = sorted(df['device'].unique())
print(f"Tổng thiết bị: {len(devices)}")
print("Danh sách thiết bị:", devices)

# --- Bước 3: Chia devices thành train và test
train_devices = devices[:20]
test_devices = devices[20:]

# --- Bước 4: Tạo tập train/test
df_train = df[df['device'].isin(train_devices)].copy()
df_test = df[df['device'].isin(test_devices)].copy()

# --- Bước 5: Chuẩn bị feature cho train và test
for dataset in [df_train, df_test]:
    if 'date' not in dataset.columns:
        dataset['date'] = pd.to_datetime(dataset['ct']).dt.date
    dataset['time_float'] = dataset['hour'] + dataset['minute'] / 60
    dataset['day_of_year'] = pd.to_datetime(dataset['date']).dt.dayofyear
    dataset['time_sin'] = np.sin(2 * np.pi * dataset['time_float'] / 24)
    dataset['time_cos'] = np.cos(2 * np.pi * dataset['time_float'] / 24)


# Loại bỏ các dòng có psun bị NaN ở train
df_train = df_train.dropna(subset=['psun'])

# --- Bước 6: One-hot encode weather_icon trên tập train
ohe = OneHotEncoder(sparse_output=False)
weather_train_encoded = ohe.fit_transform(df_train[['weather_icon']])
weather_train_df = pd.DataFrame(weather_train_encoded, 
                                columns=ohe.get_feature_names_out(['weather_icon']), 
                                index=df_train.index)
df_train = pd.concat([df_train, weather_train_df], axis=1)

# --- Bước 7: Chuẩn bị X_train, y_train
feature_cols = ['day_of_year', 'time_sin', 'time_cos'] + list(weather_train_df.columns)
X_train = df_train[feature_cols]
y_train = df_train['psun']

# --- Bước 8: Huấn luyện model
model = RandomForestRegressor(n_estimators=300, max_depth=None, min_samples_leaf=1, random_state=42)
model.fit(X_train, y_train)

# Lưu model và encoder
import joblib
joblib.dump(model, 'model_psun2.pkl')
joblib.dump(ohe, 'ohe_weather.pkl')

print("Model và encoder đã được huấn luyện và lưu.")


Xây dựng hàm xử lý data từ input để model có thể xử lý được

In [ ]:
def prepare_features(ct, weather_icon):
    """
    Tạo features đầu vào cho model dựa trên thời gian ct và weather_icon
    """
    date = ct.date()
    time_float = ct.hour + ct.minute / 60 + ct.second / 3600
    day_of_year = ct.timetuple().tm_yday
    time_sin = np.sin(2 * np.pi * time_float / 24)
    time_cos = np.cos(2 * np.pi * time_float / 24)
    
    # Chuẩn bị dataframe cho weather_icon
    weather_df = pd.DataFrame({'weather_icon': [weather_icon]})
    weather_encoded = ohe.transform(weather_df)
    weather_cols = ohe.get_feature_names_out(['weather_icon'])  # truyền tên cột, kiểu list chứa string
    weather_encoded_df = pd.DataFrame(weather_encoded, columns=weather_cols)

    
    # Tạo dataframe features đầy đủ
    features_df = pd.DataFrame({
        'day_of_year': [day_of_year],
        'time_sin': [time_sin],
        'time_cos': [time_cos]
    })
    
    features_df = pd.concat([features_df, weather_encoded_df], axis=1)
    
    return features_df

Test model: theo 6 device còn lại

In [ ]:
# Load dữ liệu đầy đủ
df = pd.read_csv('train/device_log_processed.csv')

# Lấy danh sách device từ df
test_devices = sorted(df['device'].unique())[20:]

# Lọc df_test gồm các device trong test_devices, loại bỏ dòng NaN ở cột psun
df_test = df[df['device'].isin(test_devices)].dropna(subset=['psun']).copy()

print(f"df_test shape: {df_test.shape}")

# Load model và encoder
model = joblib.load('model_psun.pkl')
ohe = joblib.load('ohe_weather.pkl')

# Tạo features list
features_list = [prepare_features(pd.to_datetime(row['ct']), row['weather_icon']) for _, row in df_test.iterrows()]

# Nối lại thành DataFrame X_test
X_test = pd.concat(features_list, ignore_index=True)

y_test = df_test['psun'].reset_index(drop=True)

# Dự đoán và đánh giá
y_pred = model.predict(X_test)
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, y_pred)
print(f"MAE trên tập test: {mae:.3f}")

# Lưu kết quả dự đoán
df_test['psun_predict'] = y_pred
df_test.to_csv('output/device_test_psun_predictions.csv', index=False)
print("File dự đoán đã được lưu tại 'output/device_test_psun_predictions.csv'")


Test model theo 1 dòng data cụ thể

In [ ]:
# Load model và encoder
model = joblib.load('model_psun.pkl')
ohe = joblib.load('ohe_weather.pkl')

# Tạo dữ liệu test thủ công
ct_test = pd.to_datetime('2025-05-13 08:10:00')
weather_icon_test = 802  # kiểu int hoặc string tùy bạn, nhớ lúc train là string thì ở đây cũng nên là string

# Tạo feature cho mẫu test
X_sample = prepare_features(ct_test, weather_icon_test)

# Dự đoán
psun_pred = model.predict(X_sample)

print(f"Dự đoán psun: {psun_pred[0]:.4f}")


Dự đoán psun tới đây là hoàn thành, giá trị dự đoán rất sát giá trị thực tế, sai lệch đa số ở thời điểm đầu ngày khi tốc độ tăng trưởng thực tế từ 0 tới 5 rất nhanh đối lập với sự tăng từ từ của model. Tuy nhiên thời gian càng trôi về giữa trưa thì dự đoán càng sát

Tìm kiếm mối liên hệ giữa psun và độ tăng của percentlt

In [ ]:
# Lấy 20 device đầu tiên
# Đã chạy thử với 5 device và ra k giống nhau, nên để 1 device cho nhanh, nếu ai muốn chắc chắn thì để 20 device. Trải nghiệm cá nhân là 3p/1device
devices_20 = sorted(df['device'].unique())[:1]

results = []

for device_id in devices_20:
    df_device = df[df['device'] == device_id].copy()
    df_device = df_device.sort_values('ct').reset_index(drop=True)

    def compute_predictions(k):
        preds = []
        prev_pred = 0
        prev_date = None

        for i, row in df_device.iterrows():
            current_date = pd.to_datetime(row['ct']).date()
            if prev_date is None or current_date != prev_date:
                prev_pred = 0
                prev_date = current_date

            pred = prev_pred + k * row['delta_minute'] * row['psun'] / 60
            preds.append(pred)
            prev_pred = pred

        return np.array(preds)

    k_values = np.linspace(0.1, 1.5, 50)
    best_k = None
    best_mae = float('inf')

    for k in k_values:
        preds = compute_predictions(k)
        mae = mean_absolute_error(df_device['plus_percentlt'], preds)
        if mae < best_mae:
            best_mae = mae
            best_k = k

    print(f'Best k on device {device_id}: {best_k:.4f} with MAE: {best_mae:.4f}')
    df_device['plus_percentlt_predict'] = compute_predictions(best_k)

    # Lưu kết quả dự đoán ra file riêng cho từng device (tuỳ chọn)
    df_device.to_csv(f'output/device_{device_id}_plus_percentlt_predict.csv', index=False)

    results.append({
        'device': device_id,
        'best_k': best_k,
        'best_mae': best_mae
    })

# Nếu muốn, gom kết quả summary lại
df_results = pd.DataFrame(results)
print(df_results)


Dựa vào mối liên hệ tìm được, xây dựng hàm dự đoán percentlt = percentlt + thời gian + weather_icon

In [ ]:
# Load model và encoder đã train
model = joblib.load('model_psun.pkl')
ohe = joblib.load('ohe_weather.pkl')

k=0.128571

# def predict_percentlt_progress(percentlt_current, ct_current, weather_icon_current):
#     """
#     Dự đoán tiến độ percentlt từ ct_current đến 18h dựa trên dự đoán psun từng 1 phút
#     Trả về thời điểm đạt 100% hoặc phần trăm tại 18h nếu chưa đạt 100%
#     """
#     delta_minute = 1
#     percentlt = percentlt_current
#     ct = ct_current
    
#     percentlt_history = [(ct, percentlt)]
    
#     while ct.hour < 18 or (ct.hour == 18 and ct.minute == 0):
#         # Chuẩn bị features dự đoán psun
#         features = prepare_features(ct, weather_icon_current)
        
#         # Dự đoán psun hiện tại
#         psun_pred = model.predict(features)[0]
        
#         # Tính percentlt mới
#         percentlt += k * delta_minute * psun_pred / 60
        
#         # Giới hạn percentlt không vượt quá 100%
#         if percentlt > 100:
#             percentlt = 100
        
#         # Cập nhật thời gian lên 1 phút
#         ct += pd.Timedelta(minutes=delta_minute)
        
#         percentlt_history.append((ct, percentlt))
        
#         # Nếu đạt 100% thì dừng
#         if percentlt >= 100:
#             break
    
#     # Kết quả
#     return percentlt_history

def predict_percentlt_progress(percentlt_current, ct_current, weather_icon_current):
    """
    Dự đoán tiến độ percentlt từ ct_current đến 19h dựa trên dự đoán psun từng 1 phút.
    Trả về:
      - thời điểm đầu tiên đạt 100% (kiểu pd.Timestamp)
      - hoặc phần trăm pin lúc 19h (kiểu float) nếu chưa đạt 100%
    """
    delta_minute = 1
    percentlt = percentlt_current
    ct = ct_current
    
    while ct.hour < 19 or (ct.hour == 19 and ct.minute == 0):
        # Chuẩn bị features dự đoán psun
        features = prepare_features(ct, weather_icon_current)
        
        # Dự đoán psun hiện tại (trả về 1 giá trị)
        psun_pred = model.predict(features)[0]
        
        # Cập nhật percentlt
        percentlt += k * delta_minute * psun_pred / 60
        
        # Giới hạn max 100%
        if percentlt >= 100:
            return ct
        
        ct += pd.Timedelta(minutes=delta_minute)
    
    # Nếu chưa đạt 100%, trả về Phần trăm lúc 19h
    return percentlt





Test cuối cùng: Đầu vào là percentlt hiện tại + thời gian hiện tại + weather_icon hiện tại

In [ ]:
import pandas as pd

percentlt_current = 74
ct_current = pd.to_datetime('2025-05-13 06:00:00')
weather_icon_current = 500
# Gọi hàm dự đoán
result = predict_percentlt_progress(percentlt_current, ct_current, weather_icon_current)

# Nếu result là thời điểm đạt 100% (Timestamp), giảm penalty
if isinstance(result, pd.Timestamp):
    result_adjusted = result
    print(f"Đạt 100% vào lúc {result_adjusted}")
else:
    print(f"Không đạt 100% trước 19h, Phần trăm lúc 19h là {result:.2f}")


Nhận xét output 12:16. Trong data thì device này percentlt = 100% tại 11:03. Sai số này tới từ việc sai số tích lũy của psun. Sẽ cải thiện bằng cách tìm ra điểm phạt phù hợp

In [ ]:
import pandas as pd

# Load data đầy đủ
df = pd.read_csv('train/device_log_processed.csv')

# Lấy danh sách thiết bị từ df
test_devices = sorted(df['device'].unique())[25:]

# Lọc ra df_test gồm các device trong test_devices, bỏ dòng psun bị NaN
df_test = df[df['device'].isin(test_devices)].dropna(subset=['psun']).copy()

# Chuyển cột thời gian sang datetime
df_test['ct'] = pd.to_datetime(df_test['ct'])

# Giới hạn 10 dòng đầu
df_test_sample = df_test.head(10).copy()

# Hàm dự đoán cho từng dòng (sửa lại phù hợp hàm mới)
def apply_predict(row):
    percentlt_current = row['percentlt']
    ct_current = row['ct']
    weather_icon_current = row['weather_icon']
    result = predict_percentlt_progress(percentlt_current, ct_current, weather_icon_current)
    if isinstance(result, pd.Timestamp):
        return f"Đạt 100% vào lúc {result}"
    else:
        return f"Không đạt 100% trước 19h, Phần trăm lúc 19h là {result:.2f}"

# Áp dụng hàm predict trên mẫu 10 dòng
df_test_sample['prediction'] = df_test_sample.apply(apply_predict, axis=1)

# In kết quả ra màn hình
print(df_test_sample[['device', 'ct', 'prediction']])

# Lưu file dự đoán (nhớ tạo thư mục output trước nếu chưa có)
df_test_sample.to_csv('output/device_test_percentlt_predictions_sample.csv', index=False)
print("Đã lưu file dự đoán mẫu 10 dòng ở 'output/device_test_percentlt_predictions_sample.csv'")


Test trên 1 file

In [ ]:
import pandas as pd

# Load data đầy đủ
df = pd.read_csv('train/device_log_processed.csv')

# Lấy danh sách thiết bị từ df
test_devices = sorted(df['device'].unique())[20:]

# Lọc ra df_test gồm các device trong test_devices, bỏ dòng psun bị NaN
df_test = df[df['device'].isin(test_devices)].dropna(subset=['psun']).copy()

# Chuyển cột thời gian sang datetime
df_test['ct'] = pd.to_datetime(df_test['ct'])

# Lọc df_test lấy các dòng có giờ từ 6 đến 18 (bao gồm 6h, chưa đến 19h)
df_filtered = df_test[(df_test['ct'].dt.hour >= 6) & (df_test['ct'].dt.hour < 18)]

# Lấy 100 dòng ngẫu nhiên từ dữ liệu đã lọc
df_test_sample = df_filtered.sample(n=100, random_state=42).copy()


# Tạo cột ngày để dễ lọc
df_test_sample['date'] = df_test_sample['ct'].dt.date

# Hàm lấy phần trăm max và thời gian tương ứng trong ngày của thiết bị
def get_max_percentlt_and_time(device, date):
    df_day = df_test[(df_test['device'] == device) & (df_test['ct'].dt.date == date)]
    if df_day.empty:
        return None, None
    idx_max = df_day['percentlt'].idxmax()
    max_percent = df_day.loc[idx_max, 'percentlt']
    max_time = df_day.loc[idx_max, 'ct']
    return max_percent, max_time

# Hàm dự đoán với penalty và so sánh thực tế
def apply_predict(row):
    percentlt_current = row['percentlt']
    ct_current = row['ct']
    weather_icon_current = row['weather_icon']

    result = predict_percentlt_progress(percentlt_current, ct_current, weather_icon_current)

    # Lấy phần trăm tối đa và thời gian thực tế
    real_max_percent, real_max_time = get_max_percentlt_and_time(row['device'], ct_current.date())

    if isinstance(result, pd.Timestamp):
        predicted_time_str = result.strftime('%Y-%m-%d %H:%M:%S')
        if real_max_percent is not None and real_max_percent >= 100:
            real_time_str = real_max_time.strftime('%Y-%m-%d %H:%M:%S')
        else:
            real_time_str = f"Phần trăm tối đa trong ngày: {real_max_percent:.2f}" if real_max_percent is not None else "Không có dữ liệu trong ngày"
        output = f"Dự đoán đạt 100% vào lúc: {predicted_time_str} | Thực tế: {real_time_str}"
    else:
        predicted_percent_str = f"Phần trăm lúc 18h: {result:.2f}"
        real_percent_str = f"Phần trăm tối đa trong ngày: {real_max_percent:.2f}" if real_max_percent is not None else "Không có dữ liệu trong ngày"
        output = f"Dự đoán: {predicted_percent_str} | Thực tế: {real_percent_str}"

    print(f"Device: {row['device']}, Time: {ct_current} -> {output}")
    return output

# Áp dụng dự đoán trên 100 dòng mẫu
df_test_sample['prediction_vs_real'] = df_test_sample.apply(apply_predict, axis=1)

# In kết quả
print(df_test_sample[['device', 'ct', 'prediction_vs_real']])

# Lưu file kết quả
df_test_sample.to_csv('output/device_test_percentlt_predictions_vs_real.csv', index=False)
print("Đã lưu file dự đoán so sánh ở 'output/device_test_percentlt_predictions_vs_real.csv'")
